In [2]:
#Works for particular font sizes only
#'blocks' is a list containing regions of image which could be paragraphs

import cv2
import numpy as np
import os
import Block

img = cv2.imread(r"C:\Users\admin\c.jpeg")	#Full path of image to be scanned
path = "C:/Users/admin/itsp_project"	#Example "C:/Users/AKHILESH/Desktop/<foldername>" Images will be stored in this folder
#path can be obtained by checking 'properties' of any other file/folder at that location

#`````````````````You don't need to change anything beyond this``````````````````````
kernel3 = np.ones((3,3), np.uint8)
kernelr1 = np.ones((7,13), np.uint8)
kernel25 = np.ones((25,25), np.uint8)

#Take image input and pre-process it to get a binary image to work on
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 5)
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel3)
eroded = cv2.erode(closed, kernelr1, iterations = 1)
inverted = cv2.bitwise_not(eroded)

#Get location of features
contours,hierarchy = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
board1 = np.zeros(eroded.shape, np.uint8)
for contour in contours:
	x,y,w,h = cv2.boundingRect(contour)
	row, col = eroded.shape
	if(h>row/2 or w>col/2 or w*h<row*col/10000 or h<30 or h>90):
		cv2.rectangle(img, (x+3,y+3), (x+w-3,y+h-3), 255, 3)
		continue
	cv2.rectangle(board1, (x+3,y+3), (x+w-3,y+h-3), 255, 3)

#Separate regions of a particular size and recognize clusters
#Store cluster in blocks[]
blocks = []
dilated = cv2.dilate(board1, kernel25, iterations = 1)
contours,hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
	x,y,w,h = cv2.boundingRect(contour)
	block = closed[y:y+h+3,x:x+w].copy()
	blocks.insert(0,block)
	cv2.rectangle(img, (x+3,y+3), (x+w-3,y+h-3), (0,0,255), 3)

#Make folders to save images and search for lines
i=1
for block in blocks:
	block_path = path + "/Block%d" %i
	os.mkdir(block_path)
	cv2.imwrite(block_path + "/block%d.jpg" %i, block)
	i+=1
	Block.findLines(block, block_path)

ModuleNotFoundError: No module named 'Block'